In [49]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import chardet

In [39]:
SCORE_DIR = "data/scores"

In [40]:
box_scores = os.listdir(SCORE_DIR)

In [41]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [51]:
box_score = box_scores[0]

utf-8


In [52]:
#cleaning the table and removing some columns/rows
def parse_html(box_score):
    with open(box_score, 'rb') as f:
        result = chardet.detect(f.read())
    detected_charset = result['encoding']
    with open(box_score, encoding=detected_charset) as f:
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [43]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    return line_score

In [44]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [45]:
def read_season_info(soup):
    #selecting first in list
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season


In [ ]:
base_cols = None
games = []
for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        #get basic and advanced states
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        #gathering the total of both basic and advance stats and making it one row
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        #gathering the max and making it a row
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        # concat both max and totals together
        summary = pd.concat([totals, maxes])

        #cleaning cols that we want and removing duplicates
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]

        summaries.append(summary)
    #turning the data frame
    summary = pd.concat(summaries, axis=1).T
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [1, 0]

    #making game in perspective of opposition
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    #combining both such that a row contains both the team and opponent scores
    full_game = pd.concat([game, game_opp], axis=1)

    #adding season to help provide information to training model
    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [56]:
games_df = pd.concat(games, ignore_index=True)

In [58]:
games_df.to_csv("nba_games.csv")